In [1]:
import json
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datasets import load_dataset
from tqdm import tqdm

sns.set_theme(style="darkgrid")
sns.set_context("talk")

/home/kyobrien/conda/envs/memorization/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load Datasets

In [ ]:
memories_path = "usvsnsp/memories-semantic-memorization-filter-results"
memories_dataset = load_dataset(memories_path,
memories_dataset

In [ ]:
pile_path = "usvsnsp/pile-semantic-memorization-filter-results"
pile_dataset = load_dataset(pile_path)
pile_dataset

In [4]:
code_path = "usvsnsp/pile-pythia-code-vs-nl-scores"
code_dataset = load_dataset(code_path)["train"].to_pandas()
code_dataset

Found cached dataset parquet (/home/kyobrien/.cache/huggingface/datasets/usvsnsp___parquet/usvsnsp--pile-pythia-code-vs-nl-scores-ec25adae8c42a5fe/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
100%|██████████| 1/1 [00:02<00:00,  2.51s/it]


,sequence_id,nl_scores
0,0,0.458252
1,1,0.480225
2,2,0.543945
3,3,0.539551
4,4,0.471191
...,...,...
146431995,146431995,0.520020
146431996,146431996,0.423096
146431997,146431997,0.541016
146431998,146431998,0.496094


In [ ]:
combined_dataframe = None
for split in tqdm(memories_dataset, desc="Loading Memories"):
    current_dataset = memories_dataset[split]
    current_frame = current_dataset.to_pandas()
    current_frame["Model"] = ".".join(split.split(".")[2:])
    current_frame["Param Count"] = split_to_param_count[current_frame["Model"].iloc[0]]
    current_frame["Deduped"] = "deduped" in split
    current_frame["Memorized"] = True
    if combined_dataframe is None:
        combined_dataframe = current_dataset.to_pandas()
    else:
        combined_dataframe = pd.concat([combined_dataframe, current_frame])

for split in tqdm(pile_dataset, desc="Loading Pile"):
    current_dataset = pile_dataset[split]
    current_frame = current_dataset.to_pandas()
    current_frame["Model"] = ".".join(split.split(".")[2:])
    current_frame["Param Count"] = split_to_param_count[current_frame["Model"].iloc[0]]
    current_frame["Deduped"] = "deduped" in split
    current_frame["Memorized"] = False
    combined_dataframe = pd.concat([combined_dataframe, current_frame])

combined_dataframe.head()

In [ ]:
# Join combined_dataframe with code_dataset on sequence_id
combined_dataframe = combined_dataframe.merge(code_dataset, on="sequence_id", how="inner")
combined_dataframe.head()

In [ ]:
sample_size = 10000
plotting_frame = combined_dataframe if sample_size is None else combined_dataframe.sample(sample_size)
display(plotting_frame.shape)

## Plot Graphs